In [4]:
import numpy as np
from cnn_utils import *
from superai.nn.layer.fc import FullyConnected
from superai.nn.layer.conv import Conv
from superai.nn.layer.pool import PoolingLayer
from superai.nn.layer.activator import Activator
from superai.nn.layer.flatten import Flatten
from superai.nn.model.nnet import Sequence
from superai.nn.optimizer.optimizer import Adam
import os
import struct
'''
from deepnet.layers import *
from deepnet.nnet import CNN
'''

X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()
X_train = X_train_orig/255.
X_train = X_train.transpose(3, 1, 2, 0)
X_test = X_test_orig/255.
X_test = X_test.transpose(3, 1, 2, 0)
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_train = Y_train.transpose(1, 0)
Y_test = convert_to_one_hot(Y_test_orig, 6).T
Y_test = Y_test.transpose(1, 0)

In [5]:
import math
def compute_samemode_pad(in_width, in_height, filter_size, strides):
    # 先确定输出维度，记住是上取整
    stride_width, stride_height = strides
    filter_width, filter_height = filter_size
    out_height = math.ceil(float(in_height) / float(stride_height))
    out_width  = math.ceil(float(in_width) / float(stride_width))

    # 上面的公式
    if (in_height % stride_height == 0):
        pad_along_height = max(filter_height - stride_height, 0)
    else:
        pad_along_height = max(filter_height - (in_height % stride_height), 0)
    if (in_width % stride_width == 0):
        pad_along_width = max(filter_width - stride_width, 0)
    else:
        pad_along_width = max(filter_width - (in_width % stride_width), 0)

    # 因为pad是在上下、左右四侧pad。所以当pi不为偶数时要分配下
    # 这里是当pi为奇数时，下侧比上侧多一，右侧比左侧多一。
    #  Note that this is different from existing libraries such as cuDNN and Caffe, which explicitly specify the number of padded pixels and always pad the same number of pixels on both sides.
    pad_top = pad_along_height // 2
    pad_bottom = pad_along_height - pad_top
    pad_left = pad_along_width // 2
    pad_right = pad_along_width - pad_left
    return (pad_top, pad_bottom, pad_left, pad_right)

In [6]:

pad = compute_samemode_pad(64, 64, (4, 4), (1, 1))
cnn_layer1 = Conv((3, 4, 4), 8, pad, (1, 1))
cnn_layer1.layer = 1
relu1 = Activator("relu")

pad = compute_samemode_pad(64, 64, (8, 8), (8, 8))
pool_layer1 = PoolingLayer((8, 8), pad, 'max', (8, 8))

pad = compute_samemode_pad(57, 57, (2, 2), (1, 1))
cnn_layer2 = Conv((8, 2, 2), 16, pad, (1, 1))
cnn_layer2.layer = 2
relu2 = Activator("relu")

pad = compute_samemode_pad(8, 8, (4, 4), (4, 4))
pool_layer2 = PoolingLayer((4, 4), pad, 'max', (4, 4))

flatten = Flatten()
dense = FullyConnected(64, 6)

model = Sequence([cnn_layer1, relu1, pool_layer1, cnn_layer2, relu2, pool_layer2, flatten, dense], learning_rate=0.001, iteration_count=100, lambd=0,
                 use_mini_batch=False, mini_batch_size=64)
'''
cnn_layer1 = Conv((3, 5, 5), 16, (2, 2, 2, 2), (1, 1))
cnn_layer1.layer = 1
relu1 = Activator("relu")
pool_layer1 = PoolingLayer((2, 2), (0,0,0,0), 'max', (2, 2))
cnn_layer2 = Conv((16, 5, 5), 20, (2, 2, 2, 2), (1, 1))
cnn_layer2.layer = 2
relu2 = Activator("relu")
pool_layer2 = PoolingLayer((2, 2), (0,0,0,0), 'max', (2, 2))

flatten = Flatten()
dense = FullyConnected(5120, 6)

model = Sequence([cnn_layer1, relu1, pool_layer1, cnn_layer2, relu2, pool_layer2, flatten, dense], learning_rate=0.01, iteration_count=300, lambd=0,
                 use_mini_batch=False, mini_batch_size=64)
'''
#model.fit(X_train[:, :, :,0:40], Y_train[:, 0:40])
adamOpt = Adam()
adamOpt.run(model, X_train[:, :, :, 0:40], Y_train[: , 0:40])


iteration1  cost:7.71834260376413, accuracy:0.17500000000000004
iteration2  cost:7.71834260376413, accuracy:0.17500000000000004
iteration3  cost:7.71834260376413, accuracy:0.17500000000000004
iteration4  cost:7.71834260376413, accuracy:0.17500000000000004
iteration5  cost:7.71834260376413, accuracy:0.17500000000000004
iteration6  cost:7.71834260376413, accuracy:0.17500000000000004


KeyboardInterrupt: 